In [1]:
from mmcv import Config
from mmseg.models import build_segmentor
from mmseg.datasets import build_dataloader, build_dataset
from mmcv.runner import load_checkpoint
import os
from mmcv.parallel import MMDataParallel
from mmseg.apis import single_gpu_test

import json
import pandas as pd
import argparse

/opt/conda/envs/mms/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/conda/envs/mms/lib/python3.8/site-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(


In [2]:
config_path = 'configs/_KDI_/uper_swin/uper_swin_large_augtest_6.py'
work_dir = 'work_dirs/uper_swin_large_augtest_6'
result_path = work_dir
best_iter = 18500

test_img_dir = '/opt/ml/input/data/mmseg/img_dir/test'
test_json = '../data/test.json'

# test_img_dir = '/opt/ml/input/data/mmseg/img_dir/val'
# test_json = '../data/val.json'

classes = [
    'Background', 'General trash', 'Paper', 'Paper pack', 'Metal', 'Glass', 'Plastic', 'Styrofoam', 'Plastic bag', 'Battery', 'Clothing'
]
seed = 8

In [3]:
cfg = Config.fromfile(config_path)

cfg.data.test.classes = classes
cfg.data.test.img_dir = test_img_dir
cfg.data.test.pipeline[1]['img_scale'] = (512, 512)
cfg.data.test.test_mode = True
cfg.data.samples_per_gpu = 4
cfg.seed = seed
cfg.gpu_ids = [1]
cfg.work_dir = work_dir
#cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2) # ???
cfg.model.train_cfg = None
    
dataset = build_dataset(cfg.data.test)
data_loader = build_dataloader(dataset, samples_per_gpu=1, workers_per_gpu=cfg.data.workers_per_gpu, dist=False, shuffle=False)

checkpoint_path = os.path.join(cfg.work_dir, f'best_mIoU_iter_{best_iter}.pth')

model = build_segmentor(cfg.model, test_cfg=cfg.get('test_cfg'))
checkpoint = load_checkpoint(model, checkpoint_path, map_location='cpu')

model.CLASSES = classes
model = MMDataParallel(model.cuda(), device_ids=[0])

output = single_gpu_test(model, data_loader)

with open(test_json, "r", encoding="utf8") as file:
    test_json_info = json.load(file)
    
file_names, preds = [], []

# for img_id, pred in enumerate(output):
#     file_names.append(test_json_info["images"][img_id]["file_name"])
#     pred = pred.reshape(1, 512, 512)
#     pred = pred.reshape((1, 256, 2, 256, 2)).max(4).max(2)
#     preds.append(' '.join(str(e) for e in pred.flatten()))

# submission = pd.read_csv('/opt/ml/input/code/submission/sample_submission.csv', index_col=None)

# # PredictionString 대입
# for file_name, string in zip(file_names, preds):
#     submission = submission.append({"image_id" : file_name, "PredictionString" : string}, 
#                                 ignore_index=True)

2022-12-30 12:18:52,351 - mmseg - INFO - Loaded 819 images
/opt/ml/input/mmsegmentation/mmseg/models/losses/cross_entropy_loss.py:235: UserWarning: Default ``avg_non_ignore`` is False, if you would like to ignore the certain label and average loss over non-ignore labels, which is the same with PyTorch official cross_entropy, set ``avg_non_ignore=True``.
  warnings.warn(


load checkpoint from local path: work_dirs/uper_swin_large_augtest_6/best_mIoU_iter_18500.pth
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 819/819, 11.7 task/s, elapsed: 70s, ETA:     0s

In [2]:
from PIL import Image
import numpy as np
from tqdm import tqdm
import shutil

In [3]:
output_ann_dir = '/opt/ml/input/data/mmseg/ann_dir/train_pseudo'
output_img_dir='/opt/ml/input/data/mmseg/img_dir/train_pseudo/'
os.makedirs(output_ann_dir, exist_ok=True)
os.makedirs(output_img_dir, exist_ok=True)

In [ ]:
# save annotatino imgs
for i in tqdm(range(len(output))):
    img = Image.fromarray(np.uint8(output[i]))
    output_ann_file = os.path.join(output_ann_dir, f'pseudo_{i:04}.png')
    img.save(output_ann_file)


In [75]:
# copy test imgs
for i in tqdm(os.listdir(test_img_dir)):
    shutil.copy(test_img_dir+'/'+i, output_img_dir+'/pseudo_'+i)

100%|██████████| 819/819 [00:00<00:00, 5965.39it/s]
